In [1]:
import cv2
import numpy as np
import time



In [2]:

# Load Yolo"
net = cv2.dnn.readNet(r"C:\Users\malik\Crop_and_weed_detection\performing_detection\data\weights\crop_weed_detection.weights", r"C:\Users\malik\Crop_and_weed_detection\performing_detection\data\cfg\crop_weed.cfg")
classes = []
with open(r"C:\Users\malik\Crop_and_weed_detection\performing_detection\data\names\obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))


# Open video capture
cap = cv2.VideoCapture(0)  # Change the argument to use a different camera index or video file

if not cap.isOpened():
    print("Failed to open video capture.")
    exit()

font = cv2.FONT_HERSHEY_PLAIN
starting_time = time.time()
frame_id = 0

while True:
    ret, frame = cap.read()

    if not ret:
        print("Failed to read frame.")
        break

    frame_id += 1
    height, width, channels = frame.shape

    # Detect objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Process detections
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.2:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    # Apply non-maximum suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.3)
    
    # Draw bounding boxes and labels
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 2, color, 2)

    # Calculate FPS and display on frame
    elapsed_time = time.time() - starting_time
    fps = frame_id / elapsed_time
    cv2.putText(frame, "FPS: " + str(round(fps, 2)), (10, 50), font, 2, (0, 0, 0), 3)

    # Display frame
    cv2.imshow("Object Detection", frame)
    
    key = cv2.waitKey(1)
    if key == 27:  # Esc key
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()
# if x == 'weed':
#     print("weed detection") 
#     if x == 'crop':
#         print("crop detection")

     # Check for key press